In [1]:
#import the random module
import random

#import pandas for df management
import pandas as pd

# import numpy
import numpy as np

#import matplotlib for plotting funtionality
import matplotlib.pyplot as plt

# Import linear regression from the SciPy stats module.
from scipy.stats import linregress

# import citipy to get closest city from lat long
from citipy import citipy

# import the datetime module from the datetime library
from datetime import datetime

# Import the API key.
from config import weather_api_key

# Import the API key.
from config import google_api_key

# Import the requests library.
import requests

import os

# import gmaps for google mapping
import gmaps

First we need to import the list of cities we created in the WeatherPy file.

In [2]:
cities_file_path = os.path.join("Weather_Data","cities.csv")
city_data_df = pd.read_csv(cities_file_path)
city_data_df.head()

,City_ID,City,Country,Date,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed
0,0,Chapais,CA,2020-07-07 21:51:27,49.78,-74.85,68.00,94,90,10.29
1,1,Ancud,CL,2020-07-07 21:51:28,-41.87,-73.82,44.01,100,20,10.29
2,2,Mataura,NZ,2020-07-07 21:51:10,-46.19,168.86,39.00,83,95,3.00
3,3,Saint Anthony,US,2020-07-07 21:51:28,45.02,-93.22,88.00,51,20,3.29
4,4,Hobart,AU,2020-07-07 21:50:34,-42.88,147.33,34.00,86,75,8.05


We should check the data types of the colums.

In [3]:
city_data_df.dtypes

City_ID         int64
City           object
Country        object
Date           object
Lat           float64
Lng           float64
Max Temp      float64
Humidity        int64
Cloudiness      int64
Wind Speed    float64
dtype: object

Now to use the gmaps api we have to give it our api key.

In [4]:
gmaps.configure(api_key=google_api_key)

Now we can make a heat map of max temps using the gmaps api.

In [11]:
# Heatmap of temperature
# Get the latitude and longitude.
locations = city_data_df[["Lat", "Lng"]]
# Assign the figure variable and format to show just one of each place
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.0)
# Get the maximum temperature.
max_temp = city_data_df["Max Temp"]
# Assign the heatmap variable, use list comprehension to remove below 0 values
heat_layer = gmaps.heatmap_layer(locations, weights=[max(temp, 0) for temp in max_temp], dissipating=False, max_intensity=300, point_radius=6)
# Add the heatmap layer.
fig.add_layer(heat_layer)
# Call the figure to plot the data.
fig

Figure(layout=FigureLayout(height='420px'))

In [12]:
# Heatmap of percent humidity
locations = city_data_df[["Lat", "Lng"]]
humidity = city_data_df["Humidity"]
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.0)
heat_layer = gmaps.heatmap_layer(locations, weights=humidity, dissipating=False, max_intensity=300, point_radius=6)

fig.add_layer(heat_layer)
# Call the figure to plot the data.
fig

Figure(layout=FigureLayout(height='420px'))

In [13]:
# Heatmap of percent cloudiness
locations = city_data_df[["Lat", "Lng"]]
clouds = city_data_df["Cloudiness"]
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.0)
heat_layer = gmaps.heatmap_layer(locations, weights = clouds, dissipating=False, max_intensity=300, point_radius=6)

fig.add_layer(heat_layer)
# Call the figure to plot the data.
fig

Figure(layout=FigureLayout(height='420px'))

Now we can start geting some input from the customer.


In [14]:
# Ask the customer to add a minimum and maximum temperature value.
min_temp = float(input("What is the minimum temperature you would like for your trip? "))
max_temp = float(input("What is the maximum temperature you would like for your trip? "))

In [17]:
print(min_temp,max_temp)

75.0 90.0


In [18]:
# Filter the dataset to find the cities that fit the criteria.
preferred_cities_df = city_data_df.loc[(city_data_df["Max Temp"] <= max_temp) & \
                                       (city_data_df["Max Temp"] >= min_temp)]
preferred_cities_df.head(10)

,City_ID,City,Country,Date,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed
3,3,Saint Anthony,US,2020-07-07 21:51:28,45.02,-93.22,88.00,51,20,3.29
7,7,Cayenne,GF,2020-07-07 21:51:29,4.93,-52.33,84.20,74,0,4.70
8,8,Albany,US,2020-07-07 21:51:29,42.60,-73.97,84.00,55,5,8.50
11,11,Sompeta,IN,2020-07-07 21:51:29,18.93,84.60,82.62,84,100,8.86
13,13,Wilkie,CA,2020-07-07 21:50:25,52.42,-108.70,81.00,64,65,5.01
14,14,Andros Town,BS,2020-07-07 21:51:30,24.70,-77.77,86.00,79,75,9.17
15,15,Georgetown,MY,2020-07-07 21:51:00,5.41,100.34,77.00,100,40,2.24
19,19,Coahuayana,MX,2020-07-07 21:51:30,18.73,-103.68,86.00,62,75,7.36
21,21,Atuona,PF,2020-07-07 21:51:31,-9.80,-139.03,79.63,76,10,14.05
24,24,Turkmenabat,TM,2020-07-07 21:51:31,39.07,63.58,78.80,27,22,13.87


In [19]:
preferred_cities_df.count()

City_ID       210
City          210
Country       210
Date          210
Lat           210
Lng           210
Max Temp      210
Humidity      210
Cloudiness    210
Wind Speed    210
dtype: int64

In [20]:
# Heatmap of temperature
# Get the latitude and longitude.
locations = preferred_cities_df[["Lat", "Lng"]]
# Assign the figure variable and format to show just one of each place
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.0)
# Get the maximum temperature.
preferred_max_temp = preferred_cities_df["Max Temp"]
# Assign the heatmap variable, use list comprehension to remove below 0 values
heat_layer = gmaps.heatmap_layer(locations, weights=[max(temp, 0) for temp in preferred_max_temp], dissipating=False, max_intensity=300, point_radius=6)
# Add the heatmap layer.
fig.add_layer(heat_layer)
# Call the figure to plot the data.
fig

Figure(layout=FigureLayout(height='420px'))

In [26]:
hotel_df = preferred_cities_df[["City","Country","Max Temp", "Lat", "Lng"]].copy()
hotel_df["Hotel Name"] = ""
hotel_df.head()

,City,Country,Max Temp,Lat,Lng,Hotel Name
3,Saint Anthony,US,88.00,45.02,-93.22,
7,Cayenne,GF,84.20,4.93,-52.33,
8,Albany,US,84.00,42.60,-73.97,
11,Sompeta,IN,82.62,18.93,84.60,
13,Wilkie,CA,81.00,52.42,-108.70,


Now we can use the nearby places API to find lodging near our city.

In [31]:
# Set parameters to search for a hotel.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": google_api_key,
    "location": "48.8566, 2.3522"}
# Use base URL to search for hotels in Paris.
base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
# Make request and get the JSON data from the search.
hotels = requests.get(base_url, params=params).json()

In [34]:
# Iterate through the DataFrame.
for index, row in hotel_df.iterrows():
    # Get the latitude and longitude.
    lat = row["Lat"]
    lng = row["Lng"]

    # Add the latitude and longitude to location key for the params dictionary.
    params["location"] = f"{lat},{lng}"

    # Use the search term: "lodging" and our latitude and longitude.
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
    # Make request and get the JSON data from the search.
    hotels = requests.get(base_url, params=params).json()
    # Grab the first hotel from the results and store the name.
    try:
        hotel_df.loc[index, "Hotel Name"] = hotels ["results"][0]["name"]
    except (IndexError):
        print("Hotel not found... skipping.")

Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.


In [35]:
hotel_df.head()

,City,Country,Max Temp,Lat,Lng,Hotel Name
3,Saint Anthony,US,88.00,45.02,-93.22,Holiday Inn Express Roseville-St. Paul
7,Cayenne,GF,84.20,4.93,-52.33,Hôtel Le Dronmi
8,Albany,US,84.00,42.60,-73.97,
11,Sompeta,IN,82.62,18.93,84.60,V.B.R A/C Lodge And Function Hall
13,Wilkie,CA,81.00,52.42,-108.70,Wilkie Motel


In [36]:
# Add a heatmap of temperature for the vacation spots and marker for each city.
locations = hotel_df[["Lat", "Lng"]]
max_temp = hotel_df["Max Temp"]
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.0)
heat_layer = gmaps.heatmap_layer(locations, weights=max_temp, 
             dissipating=False, max_intensity=300, point_radius=4)
marker_layer = gmaps.marker_layer(locations)
fig.add_layer(heat_layer)
fig.add_layer(marker_layer)
# Call the figure to plot the data.
fig

Figure(layout=FigureLayout(height='420px'))

In [38]:
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Max Temp</dt><dd>{Max Temp} °F</dd>
</dl>
"""

In [39]:
# Store the DataFrame Row.
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]

In [40]:
# Add a heatmap of temperature for the vacation spots and a pop-up marker for each city.
locations = hotel_df[["Lat", "Lng"]]
max_temp = hotel_df["Max Temp"]
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.0)
heat_layer = gmaps.heatmap_layer(locations, weights=max_temp,dissipating=False,
             max_intensity=300, point_radius=4)
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(heat_layer)
fig.add_layer(marker_layer)

# Call the figure to plot the data.
fig

Figure(layout=FigureLayout(height='420px'))